In [1]:
import tensorflow as tf
import os, logging
tf.logging.set_verbosity(tf.logging.ERROR)
import warnings
warnings.filterwarnings("ignore")
import numpy as np
from datetime import datetime

seeds = 1
tf.compat.v1.set_random_seed(seeds)
np.random.seed(seeds)

import keras.metrics
from keras import backend as K
from keras.callbacks import EarlyStopping, LambdaCallback,ReduceLROnPlateau 
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
# argparser for training the model
import argparse

def parse_args():
    parser = argparse.ArgumentParser(add_help=False)

    # dataset
    parser.add_argument('--dataset', type=str, default='physionet2012', choices=['physionet2012', 'MIMIC3'],
                        help="dataset")
    parser.add_argument('--application', type=str, default='inhos_mortality', choices=['inhos_mortality'],
                        help="dataset")
    parser.add_argument('--folds', type=list, default=[0],
                        help='folds id')
    parser.add_argument('--ffill', action='store_true',default=True,
                        help='data filling， ffill or None')
    parser.add_argument('--standardization', action='store_false',default=True,
                        help='standardization for the training dataset')
    parser.add_argument('--data_clip', action='store', default=False,
                        help='data clipping: decide the maximun and minimun value of the training dataset')
    parser.add_argument('--data_clip_min', type=float, default=-1*float('inf'),
                        help='data clipping: minimun value of the training dataset')
    parser.add_argument('--data_clip_max', type=float, default=float('inf'),
                        help='data clipping: maximun value of the training dataset')
    parser.add_argument('--dataset_mode', type=str, default='regular', choices=['regular'],
                        help="regular or irregular")
    parser.add_argument('--ffill_steps', type=int, default=48,
                        help='data filling steps')
    parser.add_argument('--max_timesteps', type=int, default=48,
                        help='Time series of at most # time steps are used. Default: 48.')

    # model
    parser.add_argument('--model', type=str, default='ELDA', choices=['ELDA','ELDA_t','ELDA_f'],
                        help="model")
    parser.add_argument('--embed_dim', type=int, default=24,
                        help="embed_dim")
    parser.add_argument('--hidden_dim', type=int, default=64,
                        help="hidden_dim")
    parser.add_argument('--clip_min', type=float, default=-3.0,
                        help="clip_min")
    parser.add_argument('--clip_max', type=float, default=3.0,
                        help="clip_max")
    parser.add_argument('--compress_dim', type=int, default=4,
                        help='compress_dim for interaction features')

    # mode
    parser.add_argument('--mode', type=str, default="train",
                        help="train or eval")
    parser.add_argument('--model_path', type=str, default="",
                        help="existing model path")
    parser.add_argument('--debug', action='store_true',default=False,
                        help='for backup')
    parser.add_argument('--random', action='store_true',default=False,
                        help='for backup')

    # training configuration
    parser.add_argument('--epoch', type=int, default=200,
                        help='Number of epochs.')
    parser.add_argument('--batch_size', type=int, default=64,
                        help='Batch size.')
    parser.add_argument('--lr', type=float, default=0.001,
                        help='Learning rate.')
    parser.add_argument('--weight_decay', type=float, default=1e-7,
                        help='decrease overfitting.')
    parser.add_argument('--patience', type=int, default=10,
                        help='patience for earlystopping.')
    parser.add_argument('--gpu', type=str, default="0,1,2",
                        help='GPU ids')

    return parser.parse_args(args=[])

args = parse_args()
args

Namespace(application='inhos_mortality', batch_size=64, clip_max=3.0, clip_min=-3.0, compress_dim=4, data_clip=False, data_clip_max=inf, data_clip_min=-inf, dataset='physionet2012', dataset_mode='regular', debug=False, embed_dim=24, epoch=200, ffill=True, ffill_steps=48, folds=[0], gpu='0,1,2', hidden_dim=64, lr=0.001, max_timesteps=48, mode='train', model='ELDA', model_path='', patience=10, random=False, standardization=True, weight_decay=1e-07)

In [3]:
# set GPU usage for tensorflow backend
if K.backend() == 'tensorflow':
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

In [4]:
if args.dataset == "physionet2012":
    from dataset.physionet2012.DataLoader import physonet2012_DataLoader
elif args.dataset == "MIMIC3":
    from dataset.MIMIC3.DataLoader import MIMIC3_DataLoader
else:
    raise FileNotFoundError("No such dataset: %s" % args.dataset)

In [5]:
# Recoders can record the results of the model in each epoch

# the results in the first three lines show the performance of the model which has the best bceloss on the validation set.
# the results in the second three lines show the performance of the model which has the best AUC-ROC on the validation set.
# the results in the last three lines show the performance of the model which has the best AUC-PR on the validation set.
import pandas as pd

class Recoder():
    def __init__(self,label,features):
        self.count = 0
        self.features = features
        self.data = {
            f: [] for f in features
        }
        self.label = label

    def insert(self,result):
        self.count += 1
        for i in self.features:
            self.data[i].append(result[i])
        return result

    def best(self,feature,mode=max):
        best_feature = mode(self.data[feature])
        index_best = self.data[feature].index(best_feature)
        return index_best

    def select_index(self,index):
        return {
            f:"%.4f" % self.data[f][index] for f in self.features
        }


class Recoders():
    def __init__(self,output_path,title,mode=2):
        self.features = ["bceloss","auroc","auprc","acc"] # ,"rec1","prec1"
        self.feature_size = len(self.features)
        if mode == 3:
            self.sets = ["train","valid","test"]
        elif mode == 2:
            self.sets = ["train", "valid"]
        self.recoders = {}
        for set in self.sets:
            self.recoders[set] = Recoder(set,self.features)
        self.count = 0
        self.output_path = output_path
        self.title = title
        self.min_score = 100


    def init_score(self,score):
        self.min_score = score

    def lower_score(self,score,delta=0):
        if score < self.min_score+delta:
            self.count = 0
            self.min_score = score
            return True
        else:
            self.count += 1
            return False

    def clean(self):
        for set in self.sets:
            self.recoders[set] = Recoder(set,self.features)
        self.count = 0
        self.min_score = 100

    def insert(self,label,result,verbose=1):
        self.recoders[label].insert(result)
        if verbose:
            log_r = "[%s] " % label
            for f in self.features:
                log_r += "%s:%.4f " % (f, result[f])
            logging.info(log_r)
            # print(log_r)

    def record_to_csv(self,i_fold=0):
        result = pd.DataFrame()
        for roc in ['bceloss','auprc','auroc']:
            tem = pd.DataFrame(columns=['Epoch']+self.features,index=self.sets)
            if roc == 'bceloss':
                func = min
            else:
                func = max
            best_epoch = self.recoders["valid"].best(roc, func)
            for set in self.sets:
                if set == "train":
                    tem.loc['train', 'Epoch'] = self.recoders["train"].count
                elif set == "valid":
                    tem.loc['valid', 'Epoch'] = best_epoch+1
                elif set == "test":
                    tem.loc['test', 'Epoch'] = self.recoders["test"].best(roc, func)+1
            for dataset in self.sets:
                for f in self.features:
                    tem.loc[dataset,f] = self.recoders[dataset].select_index(best_epoch)[f]
            result = pd.concat([result,tem],0)
        logging.info(result)
        result.to_csv(self.output_path + self.title+'%d.csv'%i_fold)


In [6]:
from sklearn import metrics

EPS = 1e-10

# binary_eval can evaluate binary prediction results
def binary_eval(y_true, predictions, verbose=1):
    predictions = np.array(predictions)
    if len(predictions.shape) == 1:
        predictions = np.stack([1 - predictions, predictions]).transpose((1, 0))

    cf = metrics.confusion_matrix(y_true, predictions.argmax(axis=1))
    if verbose:
        logging.info("confusion matrix:")
        logging.info(cf)
    cf = cf.astype(np.float32)

    acc = (cf[0][0] + cf[1][1]) / np.sum(cf)
    prec0 = cf[0][0] / (cf[0][0] + cf[1][0] + EPS)
    prec1 = cf[1][1] / (cf[1][1] + cf[0][1] + EPS)
    rec0 = cf[0][0] / (cf[0][0] + cf[0][1] + EPS)
    rec1 = cf[1][1] / (cf[1][1] + cf[1][0] + EPS)
    auroc = metrics.roc_auc_score(y_true, predictions[:, 1])

    (precisions, recalls, thresholds) = metrics.precision_recall_curve(y_true, predictions[:, 1])
    auprc = metrics.auc(recalls, precisions)
    minpse = np.max([min(x, y) for (x, y) in zip(precisions, recalls)])

    bce = []
    for i in range(len(y_true)):
        bce.append(-y_true[i] * np.log(predictions[i,1] + EPS) - (1 - y_true[i]) * np.log(1 - predictions[i,1] + EPS))
    bceloss = np.mean(bce)

    if verbose:
        logging.info("accuracy = {}".format(acc))
        logging.info("precision class 0 = {}".format(prec0))
        logging.info("precision class 1 = {}".format(prec1))
        logging.info("recall class 0 = {}".format(rec0))
        logging.info("recall class 1 = {}".format(rec1))
        logging.info("AUC of ROC = {}".format(auroc))
        logging.info("AUC of PRC = {}".format(auprc))
        logging.info("min(+P, Se) = {}".format(minpse))
        logging.info("BCEloss = {}".format(bceloss))

    return {"acc": acc,
            "prec0": prec0,
            "prec1": prec1,
            "rec0": rec0,
            "rec1": rec1,
            "auroc": auroc,
            "auprc": auprc,
            "minpse": minpse,
            "bceloss": bceloss,
            }

In [7]:
from keras import backend as K
from keras.layers import Activation, Dense, Input
from keras.layers.recurrent import GRU
from keras.layers import CuDNNGRU
from keras.models import load_model, Model
from keras.regularizers import l2

from model.keras_layers import ELDA_f,ELDA_t


def ELDA_net(info_dim,input_dim,output_dim,time_dim,args):
    input_info = Input(shape=(info_dim,))
    input_stime = Input(shape=(time_dim,))
    input_tdata = Input(shape=(time_dim, input_dim))
    input_tmask = Input(shape=(time_dim, input_dim))

    input_list = [input_info, input_stime, input_tdata, input_tmask]

    ELDA_f_layer = ELDA_f(input_dim=input_dim,
                         embedding_dim=args.embed_dim,
                         time_step=time_dim,
                         compress_dim=args.compress_dim,
                         clip_min=args.clip_min,
                         clip_max=args.clip_max,
                         )

    ELDA_t_layer = ELDA_t(time_step=time_dim,
                         hidden_dim=args.hidden_dim
                         )

    # an editable version of GRU
    # gru_layer = GRU(units=args.hidden_dim,
    #                 activation='sigmoid',
    #                 return_sequences=True,
    #                 # dropout=0.3,
    #                 # recurrent_dropout=0.3,
    #                 )
    
    # an uneditable version of GRU (for GPU)
    gru_layer =  CuDNNGRU(units=args.hidden_dim,
                 return_sequences=True,
                 )

    x = ELDA_f_layer([input_tdata, input_tmask])
    x = gru_layer(x)
    x = ELDA_t_layer(x)

    x = Dense(units=64, activation='relu',kernel_regularizer=l2(1e-4))(x)
    x = Dense(output_dim, activation='sigmoid')(x)
    output_list = [x]
    model = Model(inputs=input_list, outputs=output_list)
    return model


def ELDA_t_net(info_dim,input_dim,output_dim,time_dim,args):
    input_info = Input(shape=(info_dim,))
    input_stime = Input(shape=(time_dim,))
    input_tdata = Input(shape=(time_dim, input_dim))
    input_tmask = Input(shape=(time_dim, input_dim))

    input_list = [input_info, input_stime, input_tdata, input_tmask]

    ELDA_t_layer = ELDA_t(time_step=time_dim,
                         hidden_dim=args.hidden_dim
                         )
    # an editable version of GRU
    # gru_layer = GRU(units=args.hidden_dim,
    #                 activation='sigmoid',
    #                 return_sequences=True,
    #                 )
    
    # an uneditable version of GRU (for GPU)
    gru_layer =  CuDNNGRU(units=args.hidden_dim,
                 return_sequences=True,
                 )

    x = gru_layer(input_tdata)
    x = ELDA_t_layer(x)

    x = Dense(output_dim, activation='sigmoid')(x)
    output_list = [x]
    model = Model(inputs=input_list, outputs=output_list)
    return model

def ELDA_f_net(info_dim,input_dim,output_dim,time_dim,args):
    input_info = Input(shape=(info_dim,))
    input_stime = Input(shape=(time_dim,))
    input_tdata = Input(shape=(time_dim, input_dim))
    input_tmask = Input(shape=(time_dim, input_dim))

    input_list = [input_info, input_stime, input_tdata, input_tmask]

    ELDA_f_layer = ELDA_f(input_dim=input_dim,
                         embedding_dim=args.embed_dim,
                         time_step=time_dim,
                         compress_dim=args.compress_dim,
                         clip_min=args.clip_min,
                         clip_max=args.clip_max,
                         )
    # an editable version of GRU
    # gru_layer = GRU(units=args.hidden_dim,
    #                 activation='sigmoid',
    #                 return_sequences=False,
    #                 )
    
    # an uneditable version of GRU (for GPU)
    gru_layer =  CuDNNGRU(units=args.hidden_dim,
                 return_sequences=False,
                 )

    x = ELDA_f_layer([input_tdata, input_tmask])
    x = gru_layer(x)

    x = Dense(output_dim, activation='sigmoid')(x)
    output_list = [x]
    model = Model(inputs=input_list, outputs=output_list)
    return model

In [8]:
class main():
    def __init__(self):
        timestamp = datetime.now().strftime('%Y%m%d%H%M%S')
        self.log_path = self.set_log_path(timestamp)

        handlers = [logging.FileHandler(self.log_path + 'log_{}.txt'.format(timestamp), mode='w'), logging.StreamHandler()]
        logging.basicConfig(level=logging.INFO, datefmt='%m-%d-%y %H:%M', format='%(asctime)s:%(message)s', handlers=handlers)

        logging.info('Timestamp: {}'.format(timestamp))
        logging.info('Arguments:%s' % args)

        self.sets = ['train', 'valid', 'test']
        # sets = ['train', 'valid']

        self.recoders = Recoders(self.log_path, timestamp, len(self.sets))

        for i_fold in args.folds:
            print('{}-th fold...'.format(i_fold))
            self.dataset = self.load_dataset(i_fold)
            self.model = self.load_model()
            if args.mode == "train":
                self.train(i_fold)
            self.eval(i_fold)

    def set_log_path(self,timestamp):
        log_path = os.getcwd() + "/logs/"
        if not os.path.exists(log_path):
            os.mkdir(log_path)

        if args.debug:
            log_path += "/%s_%s_debug/" % (args.model, timestamp)
            if not os.path.exists(log_path):
                os.mkdir(log_path)
            return log_path

        log_path += "/%s/"% args.dataset
        if not os.path.exists(log_path):
            os.mkdir(log_path)

        log_path += "/%s/"% args.application
        if not os.path.exists(log_path):
            os.mkdir(log_path)

        log_path += "/%s/"% args.model
        if not os.path.exists(log_path):
            os.mkdir(log_path)

        log_path += "/%s_%s/" % (args.model, timestamp)
        if args.random:
            log_path = log_path[:-1]+"_random/"
        if not os.path.exists(log_path):
            os.mkdir(log_path)
        return log_path

    def load_dataset(self, i_fold):
        logging.info("[*] Loading dataset: %s" % args.dataset)
        if args.dataset == "physionet2012":
            dataset = physonet2012_DataLoader(args=args, fold_id=i_fold, label=args.application, debug=args.debug)
        elif args.dataset == "MIMIC3":
            dataset = MIMIC3_DataLoader(args=args, fold_id=i_fold, label=args.application, debug=args.debug)
        else:
            raise NameError(args.dataset)

        train_len = dataset.get_subset_size("train")
        valid_len = dataset.get_subset_size("valid")
        if len(self.sets) == 3:
            test_len = dataset.get_subset_size("test")
        else:
            test_len = 0
        logging.info("train: %d valid: %d test: %d" % (train_len, valid_len, test_len))
        return dataset


    def load_model(self):
        if args.dataset_mode == "regular":
            if args.model == "ELDA":
                model = ELDA_net(info_dim=self.dataset.info_dim, input_dim=self.dataset.input_dim,output_dim=self.dataset.output_dim,
                                     time_dim=48, args=args)
            elif args.model == "ELDA_t":
                model = ELDA_t_net(info_dim=self.dataset.info_dim, input_dim=self.dataset.input_dim,output_dim=self.dataset.output_dim,
                                     time_dim=48, args=args)
            elif args.model == "ELDA_f":
                model = ELDA_f_net(info_dim=self.dataset.info_dim, input_dim=self.dataset.input_dim,output_dim=self.dataset.output_dim,
                                     time_dim=48, args=args)
            else:
                raise NotImplementedError("No such model in regular mode: %s" % (args.model))

        model.summary(print_fn=logging.info)
        return model


    def eval_epoch(self, epoch, dataset, model, i_fold,save=True):
        true_y_list = [
            dataset.sub_label("train"),dataset.sub_label("valid")
        ]
        pred_y_list = [
            model.predict_generator(dataset.get_generator("train", shuffle=False, batch_size=args.batch_size, return_whole=False),
                                    steps=dataset.sub_steps("train", batch_size=args.batch_size)),
            model.predict_generator(dataset.get_generator("valid", shuffle=False, batch_size=args.batch_size, return_whole=False),
                                    steps=dataset.sub_steps("valid", batch_size=args.batch_size)),
        ]
        if len(self.sets) == 3:
            true_y_list.append(dataset.sub_label("test"))
            pred_y_list.append(model.predict_generator(dataset.get_generator("test", shuffle=False, batch_size=args.batch_size, return_whole=False),
                                                       steps=dataset.sub_steps("test", batch_size=args.batch_size)))

        logging.info("[%s Epoch %d]" % (args.model, epoch))
        for index,dataset in enumerate(self.sets):
            result = binary_eval(np.reshape(true_y_list[index],(-1)), np.reshape(pred_y_list[index],(-1)), 0)
            self.recoders.insert(dataset, result)
            if dataset == "valid" and self.recoders.lower_score(result['bceloss']) and save:
                logging.info("[*] model_%d.h5 saved "%i_fold)
                model.save(self.log_path+'model_%d.h5'%i_fold)



    def train(self, i_fold):
        optimizer = Adam(learning_rate=args.lr,decay=1e-5)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, mode='auto')
        self.model.compile(optimizer=optimizer, loss='binary_crossentropy',metrics=[keras.metrics.BinaryCrossentropy()])

        self.model.fit_generator(
            generator=self.dataset.get_generator("train", shuffle=True, batch_size=args.batch_size, return_whole=True),
            steps_per_epoch=self.dataset.sub_steps("train", batch_size=args.batch_size),
            epochs=args.epoch,
            verbose=1,
            validation_data=self.dataset.get_generator("valid", shuffle=False, batch_size=args.batch_size, return_whole=True),
            validation_steps=self.dataset.sub_steps("valid", batch_size=args.batch_size),
            callbacks=[
                LambdaCallback(on_epoch_end=lambda epoch, logs: self.eval_epoch(epoch + 1, self.dataset, self.model, i_fold)),
                EarlyStopping(monitor="val_binary_crossentropy",patience=args.patience,mode="min"),
                reduce_lr,
            ]
        )
        self.recoders.record_to_csv(i_fold)
        self.recoders.clean()


    def eval(self, i_fold):
        if args.mode == "eval":
            if args.model_path != "" and os.path.exists(args.model_path):
                self.model.load_weights(args.model_path)
            else:
                raise FileNotFoundError("There is no such file for the model: %s" % args.model_path)

        elif args.mode == "train":
            self.model.load_weights(self.log_path+'model_%d.h5'%i_fold)

        logging.info("[Eval]")
        self.eval_epoch(0, self.dataset, self.model, i_fold, False)


In [9]:
main()

07-23-21 20:38:Timestamp: 20210723203807
07-23-21 20:38:Arguments:Namespace(application='inhos_mortality', batch_size=64, clip_max=3.0, clip_min=-3.0, compress_dim=4, data_clip=False, data_clip_max=inf, data_clip_min=-inf, dataset='physionet2012', dataset_mode='regular', debug=False, embed_dim=24, epoch=200, ffill=True, ffill_steps=48, folds=[0], gpu='0,1,2', hidden_dim=64, lr=0.001, max_timesteps=48, mode='train', model='ELDA', model_path='', patience=10, random=False, standardization=True, weight_decay=1e-07)
07-23-21 20:38:[*] Loading dataset: physionet2012
07-23-21 20:38:fold path: inhos_mortality_folds.npz
07-23-21 20:38:train: [150491 143937 148710 140049 156691]
  0%|          | 30/9600 [00:00<00:32, 295.92it/s]

0-th fold...


100%|██████████| 1200/1200 [00:04<00:00, 281.71it/s]
07-23-21 20:38:train: 9600 valid: 1200 test: 1200
07-23-21 20:38:Model: "model_1"
07-23-21 20:38:__________________________________________________________________________________________________
07-23-21 20:38:Layer (type)                    Output Shape         Param #     Connected to                     
07-23-21 20:38:==================================================================================================
07-23-21 20:38:input_3 (InputLayer)            (None, 48, 37)       0                                            
07-23-21 20:38:__________________________________________________________________________________________________
07-23-21 20:38:input_4 (InputLayer)            (None, 48, 37)       0                                            
07-23-21 20:38:__________________________________________________________________________________________________
07-23-21 20:38:elda_f_1 (ELDA_f)               (None, 48, 148)     

Epoch 1/200
150/150 [==============================] - 8s 54ms/step - loss: 0.4012 - binary_crossentropy: 0.3948 - val_loss: 0.4393 - val_binary_crossentropy: 0.3397


07-23-21 20:39:[ELDA Epoch 1]
07-23-21 20:39:[train] bceloss:0.3291 auroc:0.8162 auprc:0.4474 acc:0.8649 
07-23-21 20:39:[valid] bceloss:0.3397 auroc:0.7888 auprc:0.4463 acc:0.8675 
07-23-21 20:39:[*] model_0.h5 saved 
07-23-21 20:39:[test] bceloss:0.3347 auroc:0.8080 auprc:0.4327 acc:0.8667 


...

07-23-21 20:44:[ELDA Epoch 35]
07-23-21 20:44:[train] bceloss:0.2503 auroc:0.8996 auprc:0.6660 acc:0.8976 
07-23-21 20:44:[valid] bceloss:0.2923 auroc:0.8563 auprc:0.5631 acc:0.8867 
07-23-21 20:44:[test] bceloss:0.2736 auroc:0.8867 auprc:0.5784 acc:0.8833 


Epoch 36/200
150/150 [==============================] - 7s 44ms/step - loss: 0.2535 - binary_crossentropy: 0.2512 - val_loss: 0.4049 - val_binary_crossentropy: 0.2923


07-23-21 20:44:[ELDA Epoch 36]
07-23-21 20:44:[train] bceloss:0.2501 auroc:0.8998 auprc:0.6671 acc:0.8976 
07-23-21 20:44:[valid] bceloss:0.2923 auroc:0.8556 auprc:0.5612 acc:0.8867 
07-23-21 20:44:[test] bceloss:0.2729 auroc:0.8867 auprc:0.5780 acc:0.8808 


Epoch 37/200
150/150 [==============================] - 7s 44ms/step - loss: 0.2531 - binary_crossentropy: 0.2509 - val_loss: 0.4110 - val_binary_crossentropy: 0.2935


07-23-21 20:44:[ELDA Epoch 37]
07-23-21 20:44:[train] bceloss:0.2498 auroc:0.8999 auprc:0.6669 acc:0.8968 
07-23-21 20:44:[valid] bceloss:0.2935 auroc:0.8562 auprc:0.5649 acc:0.8850 
07-23-21 20:44:[test] bceloss:0.2741 auroc:0.8873 auprc:0.5798 acc:0.8825 
07-23-21 20:44:      Epoch bceloss   auroc   auprc     acc
train    37  0.2555  0.8961  0.6556  0.8947
valid    27  0.2892  0.8575  0.5681  0.8892
test     25  0.2723  0.8869  0.5770  0.8792
train    37  0.2581  0.8931  0.6449  0.8942
valid    23  0.2892  0.8581  0.5692  0.8908
test     24  0.2731  0.8863  0.5792  0.8800
train    37  0.2599  0.8918  0.6409  0.8948
valid    22  0.2892  0.8581  0.5691  0.8908
test     24  0.2735  0.8855  0.5754  0.8792
07-23-21 20:44:[Eval]
07-23-21 20:44:[ELDA Epoch 0]
07-23-21 20:44:[train] bceloss:0.2555 auroc:0.8961 auprc:0.6556 acc:0.8947 
07-23-21 20:44:[valid] bceloss:0.2892 auroc:0.8575 auprc:0.5681 acc:0.8892 
07-23-21 20:44:[test] bceloss:0.2723 auroc:0.8869 auprc:0.5770 acc:0.8792 
